In [1]:
using GeneralNN
using DelimitedFiles
using BenchmarkTools

[ Info: Precompiling GeneralNN [47e0c118-78ee-11ef-3e02-d9224f55ee60] (cache misses: wrong dep version loaded (2))
[ Info: Precompiling IJuliaExt [2f4121a4-3b3a-5ce6-9c5e-1f2673ce168a] (cache misses: wrong dep version loaded (2))
[ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf] (cache misses: wrong dep version loaded (2))


In [2]:
pwd()

"/Users/lewis/code/nn by hand/notebooks"

In [3]:
includet("../src/conv_net_funcs.jl")

In [4]:
includet("../src/colconv.jl")

In [5]:
includet("../src/simple_conv.jl")


In [6]:
for (mod, file) in Revise.included_files
    Revise.track(mod, file)
end

#### Load some test data to play with

In [7]:
edg = readdlm("../tstdata/edg.csv", ',')

6×6 Matrix{Float64}:
 10.0  10.0  10.0  0.0  0.0  0.0
 10.0  10.0  10.0  0.0  0.0  0.0
 10.0  10.0  10.0  0.0  0.0  0.0
 10.0  10.0  10.0  0.0  0.0  0.0
 10.0  10.0  10.0  0.0  0.0  0.0
 10.0  10.0  10.0  0.0  0.0  0.0

In [8]:
edg_6by6_by3_by1 = reshape(cat(edg, edg,edg,dims=3),(6,6,3,1))

6×6×3×1 Array{Float64, 4}:
[:, :, 1, 1] =
 10.0  10.0  10.0  0.0  0.0  0.0
 10.0  10.0  10.0  0.0  0.0  0.0
 10.0  10.0  10.0  0.0  0.0  0.0
 10.0  10.0  10.0  0.0  0.0  0.0
 10.0  10.0  10.0  0.0  0.0  0.0
 10.0  10.0  10.0  0.0  0.0  0.0

[:, :, 2, 1] =
 10.0  10.0  10.0  0.0  0.0  0.0
 10.0  10.0  10.0  0.0  0.0  0.0
 10.0  10.0  10.0  0.0  0.0  0.0
 10.0  10.0  10.0  0.0  0.0  0.0
 10.0  10.0  10.0  0.0  0.0  0.0
 10.0  10.0  10.0  0.0  0.0  0.0

[:, :, 3, 1] =
 10.0  10.0  10.0  0.0  0.0  0.0
 10.0  10.0  10.0  0.0  0.0  0.0
 10.0  10.0  10.0  0.0  0.0  0.0
 10.0  10.0  10.0  0.0  0.0  0.0
 10.0  10.0  10.0  0.0  0.0  0.0
 10.0  10.0  10.0  0.0  0.0  0.0

In [9]:
v_edge_fil = readdlm("../tstdata/v_edge_fil.csv", ',')

3×3 Matrix{Float64}:
 1.0  0.0  -1.0
 1.0  0.0  -1.0
 1.0  0.0  -1.0

In [10]:
h_edge_fil = readdlm("../tstdata/h_edge_fil.csv", ',')

3×3 Matrix{Float64}:
  1.0   1.0   1.0
  0.0   0.0   0.0
 -1.0  -1.0  -1.0

In [11]:
edg2_fil = readdlm("../tstdata/edg2_fil.csv", ',')

6×6 Matrix{Float64}:
 10.0  10.0  10.0   0.0   0.0   0.0
 10.0  10.0  10.0   0.0   0.0   0.0
 10.0  10.0  10.0   0.0   0.0   0.0
  0.0   0.0   0.0  10.0  10.0  10.0
  0.0   0.0   0.0  10.0  10.0  10.0
  0.0   0.0   0.0  10.0  10.0  10.0

In [12]:
same_fil = readdlm("../tstdata/same_fil.csv", ',')

3×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  0.0

In [13]:
function multiply_fil(x)
x .* [0 0 0;
    0 1 0;
    0 0 0]
end

multiply_fil (generic function with 1 method)

In [14]:
multiply_fil(2.0)

3×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  2.0  0.0
 0.0  0.0  0.0

In [15]:
x = readdlm("../tstdata/x.csv", ',')

6×6 Matrix{Float64}:
 3.0  0.0  1.0  2.0  7.0  4.0
 1.0  5.0  8.0  9.0  3.0  1.0
 2.0  7.0  2.0  5.0  1.0  3.0
 0.0  1.0  3.0  1.0  7.0  8.0
 4.0  2.0  1.0  6.0  2.0  8.0
 2.0  4.0  5.0  2.0  3.0  9.0

In [16]:
schorr = readdlm("../tstdata/schorr_fil.csv", ',')

3×3 Matrix{Float64}:
  3.0  0.0   -3.0
 10.0  0.0  -10.0
  3.0  0.0   -3.0

In [17]:
sobel = readdlm("../tstdata/sobel_fil.csv", ',')

3×3 Matrix{Float64}:
 1.0  0.0  -1.0
 2.0  0.0  -2.0
 1.0  0.0  -1.0

In [18]:
fiva = collect(reshape(1:25, 5, 5)')
sixa = collect(reshape(1:36, 6, 6)')

6×6 Matrix{Int64}:
  1   2   3   4   5   6
  7   8   9  10  11  12
 13  14  15  16  17  18
 19  20  21  22  23  24
 25  26  27  28  29  30
 31  32  33  34  35  36

In [19]:
res6, locs6 = pooling(sixa, stridedims=(2,2))
res5, locs5, = pooling(fiva)

([7.0 9.0; 17.0 19.0;;;], CartesianIndex{2}[CartesianIndex(2, 2) CartesianIndex(2, 2); CartesianIndex(2, 2) CartesianIndex(2, 2);;;])

In [20]:
res5

2×2×1 Array{Float64, 3}:
[:, :, 1] =
  7.0   9.0
 17.0  19.0

In [21]:
locs5

2×2×1 Array{CartesianIndex{2}, 3}:
[:, :, 1] =
 CartesianIndex(2, 2)  CartesianIndex(2, 2)
 CartesianIndex(2, 2)  CartesianIndex(2, 2)

In [22]:
foo6 = unpooling(res6, locs6, prevdims = (6,6,1), stridedims=(2,2))

6×6×1 Array{Float64, 3}:
[:, :, 1] =
 0.0   0.0  0.0   0.0  0.0   0.0
 0.0   8.0  0.0  10.0  0.0  12.0
 0.0   0.0  0.0   0.0  0.0   0.0
 0.0  20.0  0.0  22.0  0.0  24.0
 0.0   0.0  0.0   0.0  0.0   0.0
 0.0  32.0  0.0  34.0  0.0  36.0

In [23]:
foo5 = unpooling(res5, locs5, prevdims=(5,5,1))

5×5×1 Array{Float64, 3}:
[:, :, 1] =
 0.0   0.0  0.0   0.0  0.0
 0.0   7.0  0.0   9.0  0.0
 0.0   0.0  0.0   0.0  0.0
 0.0  17.0  0.0  19.0  0.0
 0.0   0.0  0.0   0.0  0.0

In [24]:
for xi in 1:1:5-1
println(xi)
end

1
2
3
4


#### Run some functions

In [25]:
convolve_multi(edg, v_edge_fil, [1])

stride = 1
(typeof(stride), stride) = (Int64, 1)
(typeof(stride), stride) = (Int64, 1)


6×6×1 Array{Float64, 3}:
[:, :, 1] =
 -19.0  1.0  21.0  21.0  1.0  1.0
 -29.0  1.0  31.0  31.0  1.0  1.0
 -29.0  1.0  31.0  31.0  1.0  1.0
 -29.0  1.0  31.0  31.0  1.0  1.0
 -29.0  1.0  31.0  31.0  1.0  1.0
 -19.0  1.0  21.0  21.0  1.0  1.0

In [26]:
dopad(edg,(1,1,1,1))

8×8 Matrix{Float64}:
 0.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0
 0.0  10.0  10.0  10.0  0.0  0.0  0.0  0.0
 0.0  10.0  10.0  10.0  0.0  0.0  0.0  0.0
 0.0  10.0  10.0  10.0  0.0  0.0  0.0  0.0
 0.0  10.0  10.0  10.0  0.0  0.0  0.0  0.0
 0.0  10.0  10.0  10.0  0.0  0.0  0.0  0.0
 0.0  10.0  10.0  10.0  0.0  0.0  0.0  0.0
 0.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0

In [27]:
pooling(edg)

([10.0 10.0 0.0; 10.0 10.0 0.0; 10.0 10.0 0.0;;;], CartesianIndex{2}[CartesianIndex(1, 1) CartesianIndex(1, 1) CartesianIndex(1, 1); CartesianIndex(1, 1) CartesianIndex(1, 1) CartesianIndex(1, 1); CartesianIndex(1, 1) CartesianIndex(1, 1) CartesianIndex(1, 1);;;])

In [28]:
convolve_multi(edg, multiply_fil(3.0), [1])

stride = 1
(typeof(stride), stride) = (Int64, 1)
(typeof(stride), stride) = (Int64, 1)


6×6×1 Array{Float64, 3}:
[:, :, 1] =
 31.0  31.0  31.0  1.0  1.0  1.0
 31.0  31.0  31.0  1.0  1.0  1.0
 31.0  31.0  31.0  1.0  1.0  1.0
 31.0  31.0  31.0  1.0  1.0  1.0
 31.0  31.0  31.0  1.0  1.0  1.0
 31.0  31.0  31.0  1.0  1.0  1.0

In [29]:
padding_same(edg, v_edge_fil, stride=1)

(1, 1, 1, 1)

In [30]:
foo = padding_same(edg, v_edge_fil, stride=1)

(1, 1, 1, 1)

In [31]:
pad_r1, pad_r2 = padding_same(edg, v_edge_fil, stride=1)

(1, 1, 1, 1)

In [32]:
tst20by3img = zeros(20,20,3)
tst20by3img[:,:,1] .= ones(20,20)
tst20by3img[:,:,2] .= fill(2, 20,20)
tst20by3img[:,:,3] .= fill(3,20,20)
tst20by3img

20×20×3 Array{Float64, 3}:
[:, :, 1] =
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0 

In [33]:
tst18img = rand(18,18)

18×18 Matrix{Float64}:
 0.433972   0.727757   0.186255   …  0.9173     0.0737521  0.964377
 0.0294257  0.0608047  0.384322      0.362926   0.487544   0.0585121
 0.515402   0.366928   0.375159      0.0589943  0.193696   0.967343
 0.327537   0.974179   0.0107704     0.0550266  0.871425   0.396937
 0.84858    0.56658    0.0779224     0.233871   0.589388   0.814065
 0.811784   0.730457   0.455898   …  0.0021287  0.0714399  0.348278
 0.139634   0.622982   0.979274      0.860846   0.873954   0.684054
 0.978795   0.39101    0.442883      0.488955   0.825551   0.687747
 0.51633    0.290917   0.802172      0.216126   0.65301    0.766515
 0.195613   0.103164   0.573345      0.981229   0.521362   0.7308
 0.609827   0.1672     0.690273   …  0.0263714  0.610881   0.179506
 0.170923   0.925889   0.24885       0.29892    0.891393   0.610055
 0.102836   0.698026   0.702232      0.495421   0.286726   0.719913
 0.073249   0.44365    0.612868      0.654246   0.374134   0.740456
 0.285649   0.547978   0.1

In [34]:
randfil = rand(5,5)

5×5 Matrix{Float64}:
 0.938453  0.97719     0.751403  0.924256  0.171873
 0.4622    0.357874    0.345763  0.943722  0.0194783
 0.560886  0.107025    0.494387  0.829282  0.693577
 0.458036  0.342218    0.51568   0.358803  0.920442
 0.159618  0.00554671  0.154229  0.963905  0.985727

In [35]:
minifil = rand(2,2)

2×2 Matrix{Float64}:
 0.569496  0.141994
 0.616643  0.781731

In [36]:
new_img_size(tst20by3img, randfil, padding=:same)

(20, 20, (2, 2, 2, 2))

In [37]:
new_img_size(tst18img, minifil, stride=2, padding=:none)

(9, 9, (0, 0, 0, 0))

#### Multiple input channels, multiple filters convolutional network with 2 layers

In [38]:
filA3by3by5 = zeros(3,3,5)
filA3by3by5[:,:,1] = fill(0.5, 3,3)
filA3by3by5[:,:,2] = fill(0.05, 3,3)
filA3by3by5[:,:,3] = fill(0.005, 3,3)
filA3by3by5[:,:,4] = fill(0.0005, 3,3)
filA3by3by5[:,:,5] = fill(0.00005, 3,3)
filA3by3by5

3×3×5 Array{Float64, 3}:
[:, :, 1] =
 0.5  0.5  0.5
 0.5  0.5  0.5
 0.5  0.5  0.5

[:, :, 2] =
 0.05  0.05  0.05
 0.05  0.05  0.05
 0.05  0.05  0.05

[:, :, 3] =
 0.005  0.005  0.005
 0.005  0.005  0.005
 0.005  0.005  0.005

[:, :, 4] =
 0.0005  0.0005  0.0005
 0.0005  0.0005  0.0005
 0.0005  0.0005  0.0005

[:, :, 5] =
 5.0e-5  5.0e-5  5.0e-5
 5.0e-5  5.0e-5  5.0e-5
 5.0e-5  5.0e-5  5.0e-5

In [39]:
biasA = fill(0.2, 5)

5-element Vector{Float64}:
 0.2
 0.2
 0.2
 0.2
 0.2

In [40]:
filB3by3by5 = zeros(3,3,5)
filB3by3by5[:,:,1] = fill(0.1, 3,3)
filB3by3by5[:,:,2] = fill(0.01, 3,3)
filB3by3by5[:,:,3] = fill(0.001, 3,3)
filB3by3by5[:,:,4] = fill(0.0001, 3,3)
filB3by3by5[:,:,5] = fill(0.00001, 3,3)
filB3by3by5

3×3×5 Array{Float64, 3}:
[:, :, 1] =
 0.1  0.1  0.1
 0.1  0.1  0.1
 0.1  0.1  0.1

[:, :, 2] =
 0.01  0.01  0.01
 0.01  0.01  0.01
 0.01  0.01  0.01

[:, :, 3] =
 0.001  0.001  0.001
 0.001  0.001  0.001
 0.001  0.001  0.001

[:, :, 4] =
 0.0001  0.0001  0.0001
 0.0001  0.0001  0.0001
 0.0001  0.0001  0.0001

[:, :, 5] =
 1.0e-5  1.0e-5  1.0e-5
 1.0e-5  1.0e-5  1.0e-5
 1.0e-5  1.0e-5  1.0e-5

In [41]:
biasB = fill(0.05, 5)

5-element Vector{Float64}:
 0.05
 0.05
 0.05
 0.05
 0.05

##### check dopad with multiple channel image

In [42]:
allpads = padding_same(tst20by3img, filA3by3by5)

(1, 1, 1, 1)

In [43]:
dopad(tst20by3img, allpads)

22×22×3 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  0.0
 0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  0.0
 0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  0.0
 0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  0.0
 0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  0.0
 0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  0.0
 0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  0.0
 0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  0.0
 0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  0.0
 0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  0.0
 0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  0.0
 0.0  1.0  1.0  1.0  1.0 

In [44]:
conv1out = convolve_multi(tst20by3img, filA3by3by5, biasA)

stride = 1
(typeof(stride), stride) = (Int64, 1)
(typeof(stride), stride) = (Int64, 1)


20×20×1 Array{Float64, 3}:
[:, :, 1] =
 2.66  3.89   3.89   3.89   3.89   3.89   …  3.89   3.89   3.89   3.89   2.66
 3.89  5.735  5.735  5.735  5.735  5.735     5.735  5.735  5.735  5.735  3.89
 3.89  5.735  5.735  5.735  5.735  5.735     5.735  5.735  5.735  5.735  3.89
 3.89  5.735  5.735  5.735  5.735  5.735     5.735  5.735  5.735  5.735  3.89
 3.89  5.735  5.735  5.735  5.735  5.735     5.735  5.735  5.735  5.735  3.89
 3.89  5.735  5.735  5.735  5.735  5.735  …  5.735  5.735  5.735  5.735  3.89
 3.89  5.735  5.735  5.735  5.735  5.735     5.735  5.735  5.735  5.735  3.89
 3.89  5.735  5.735  5.735  5.735  5.735     5.735  5.735  5.735  5.735  3.89
 3.89  5.735  5.735  5.735  5.735  5.735     5.735  5.735  5.735  5.735  3.89
 3.89  5.735  5.735  5.735  5.735  5.735     5.735  5.735  5.735  5.735  3.89
 3.89  5.735  5.735  5.735  5.735  5.735  …  5.735  5.735  5.735  5.735  3.89
 3.89  5.735  5.735  5.735  5.735  5.735     5.735  5.735  5.735  5.735  3.89
 3.89  5.735  5.735  5.73

In [45]:
v_edge_fil_3_3_3_1 = cat(v_edge_fil, v_edge_fil, v_edge_fil,dims=3)

3×3×3 Array{Float64, 3}:
[:, :, 1] =
 1.0  0.0  -1.0
 1.0  0.0  -1.0
 1.0  0.0  -1.0

[:, :, 2] =
 1.0  0.0  -1.0
 1.0  0.0  -1.0
 1.0  0.0  -1.0

[:, :, 3] =
 1.0  0.0  -1.0
 1.0  0.0  -1.0
 1.0  0.0  -1.0

In [46]:
v_edge_fil_3_3_3_1 = reshape(v_edge_fil_3_3_3_1, (3,3,3,1))

3×3×3×1 Array{Float64, 4}:
[:, :, 1, 1] =
 1.0  0.0  -1.0
 1.0  0.0  -1.0
 1.0  0.0  -1.0

[:, :, 2, 1] =
 1.0  0.0  -1.0
 1.0  0.0  -1.0
 1.0  0.0  -1.0

[:, :, 3, 1] =
 1.0  0.0  -1.0
 1.0  0.0  -1.0
 1.0  0.0  -1.0

In [47]:
size(edg_6by6_by3_by1)

(6, 6, 3, 1)

In [48]:
size(conv_col_out)

LoadError: UndefVarError: `conv_col_out` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [49]:
clay = ConvLayer(v_edge_fil_3_3_3_1,fill(0.2,3),1,1)

ConvLayer([1.0 0.0 -1.0; 1.0 0.0 -1.0; 1.0 0.0 -1.0;;; 1.0 0.0 -1.0; 1.0 0.0 -1.0; 1.0 0.0 -1.0;;; 1.0 0.0 -1.0; 1.0 0.0 -1.0; 1.0 0.0 -1.0;;;;], [0.2, 0.2, 0.2], 1, 1)

In [51]:
size(clay.weights)

(3, 3, 3, 1)

In [52]:
@benchmark forward(clay, edg_6by6_by3_by1)

BenchmarkTools.Trial: 10000 samples with 6 evaluations.
 Range (min … max):  5.431 μs … 609.035 μs  ┊ GC (min … max): 0.00% … 98.28%
 Time  (median):     5.889 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.355 μs ±  10.852 μs  ┊ GC (mean ± σ):  5.64% ±  3.61%

          ██▄                                                  
  ▂██▄▃▃▄████▇▄▂▂▂▂▂▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  5.43 μs         Histogram: frequency by time        8.35 μs <

 Memory estimate: 10.91 KiB, allocs estimate: 12.

In [81]:
@benchmark x_col, w_col, bias, out_channels, batch_size, out_height, out_width = prep_forward(clay, edg_6by6_by3_by1)

BenchmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):  4.905 μs … 244.351 μs  ┊ GC (min … max): 0.00% … 96.85%
 Time  (median):     5.327 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.678 μs ±   7.245 μs  ┊ GC (mean ± σ):  4.86% ±  3.72%

   ▁▃     ██▄▂                                                 
  ▂██▆▃▃▄█████▆▄▃▂▂▂▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  4.9 μs          Histogram: frequency by time        7.58 μs <

 Memory estimate: 9.73 KiB, allocs estimate: 8.

In [79]:
@benchmark forward(x_col, w_col, bias, out_channels, batch_size, out_height, out_width)

BenchmarkTools.Trial: 10000 samples with 136 evaluations.
 Range (min … max):  721.507 ns …   7.042 μs  ┊ GC (min … max): 0.00% … 86.51%
 Time  (median):     738.051 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   773.808 ns ± 323.979 ns  ┊ GC (mean ± σ):  2.56% ±  5.37%

  ▁▆██▇▆▅▃▃▄▃▃▂▂▂▂▁▁▁▁▁▁ ▁                                      ▂
  █████████████████████████████▇█▇█▇▇█▇█▆▆▇▇▇▆▆▇▇▇▇▇▆▆▅▆▄▆▅▅▅▅▅ █
  722 ns        Histogram: log(frequency) by time        964 ns <

 Memory estimate: 1.34 KiB, allocs estimate: 9.

In [78]:
img_out1 = forward(x_col, w_col, bias, out_channels, batch_size, out_height, out_width)

6×6×1×1 Array{Float64, 4}:
[:, :, 1, 1] =
 -59.8  0.2  60.2  60.2  0.2  0.2
 -89.8  0.2  90.2  90.2  0.2  0.2
 -89.8  0.2  90.2  90.2  0.2  0.2
 -89.8  0.2  90.2  90.2  0.2  0.2
 -89.8  0.2  90.2  90.2  0.2  0.2
 -59.8  0.2  60.2  60.2  0.2  0.2

In [55]:
img_out1 = forward(clay, edg_6by6_by3_by1)

6×6×1×1 Array{Float64, 4}:
[:, :, 1, 1] =
 -59.8  0.2  60.2  60.2  0.2  0.2
 -89.8  0.2  90.2  90.2  0.2  0.2
 -89.8  0.2  90.2  90.2  0.2  0.2
 -89.8  0.2  90.2  90.2  0.2  0.2
 -89.8  0.2  90.2  90.2  0.2  0.2
 -59.8  0.2  60.2  60.2  0.2  0.2

In [56]:
size(img_out1)

(6, 6, 1, 1)

In [57]:
@show size(x), size(x_col), size(w_col)

(size(x), size(x_col), size(w_col)) = ((6, 6), (27, 36), (27,))


((6, 6), (27, 36), (27,))

In [58]:
img_out1 = col2im(x[:,:,1,1], size(x), 3, 1, 0)

BoundsError: BoundsError: attempt to access Tuple{Int64, Int64} at index [3]

In [59]:
activ1 = similar(conv1out)
GeneralNN.relu!(activ1, conv1out)

20×20×1 Array{Float64, 3}:
[:, :, 1] =
 2.66  3.89   3.89   3.89   3.89   3.89   …  3.89   3.89   3.89   3.89   2.66
 3.89  5.735  5.735  5.735  5.735  5.735     5.735  5.735  5.735  5.735  3.89
 3.89  5.735  5.735  5.735  5.735  5.735     5.735  5.735  5.735  5.735  3.89
 3.89  5.735  5.735  5.735  5.735  5.735     5.735  5.735  5.735  5.735  3.89
 3.89  5.735  5.735  5.735  5.735  5.735     5.735  5.735  5.735  5.735  3.89
 3.89  5.735  5.735  5.735  5.735  5.735  …  5.735  5.735  5.735  5.735  3.89
 3.89  5.735  5.735  5.735  5.735  5.735     5.735  5.735  5.735  5.735  3.89
 3.89  5.735  5.735  5.735  5.735  5.735     5.735  5.735  5.735  5.735  3.89
 3.89  5.735  5.735  5.735  5.735  5.735     5.735  5.735  5.735  5.735  3.89
 3.89  5.735  5.735  5.735  5.735  5.735     5.735  5.735  5.735  5.735  3.89
 3.89  5.735  5.735  5.735  5.735  5.735  …  5.735  5.735  5.735  5.735  3.89
 3.89  5.735  5.735  5.735  5.735  5.735     5.735  5.735  5.735  5.735  3.89
 3.89  5.735  5.735  5.73

In [60]:
conv2out = convolve_multi(activ1, filB3by3by5, biasB, padding=:none)

18×18×1 Array{Float64, 3}:
[:, :, 1] =
 4.166  4.658   4.658   4.658   4.658   …  4.658   4.658   4.658   4.166
 4.658  5.2115  5.2115  5.2115  5.2115     5.2115  5.2115  5.2115  4.658
 4.658  5.2115  5.2115  5.2115  5.2115     5.2115  5.2115  5.2115  4.658
 4.658  5.2115  5.2115  5.2115  5.2115     5.2115  5.2115  5.2115  4.658
 4.658  5.2115  5.2115  5.2115  5.2115     5.2115  5.2115  5.2115  4.658
 4.658  5.2115  5.2115  5.2115  5.2115  …  5.2115  5.2115  5.2115  4.658
 4.658  5.2115  5.2115  5.2115  5.2115     5.2115  5.2115  5.2115  4.658
 4.658  5.2115  5.2115  5.2115  5.2115     5.2115  5.2115  5.2115  4.658
 4.658  5.2115  5.2115  5.2115  5.2115     5.2115  5.2115  5.2115  4.658
 4.658  5.2115  5.2115  5.2115  5.2115     5.2115  5.2115  5.2115  4.658
 4.658  5.2115  5.2115  5.2115  5.2115  …  5.2115  5.2115  5.2115  4.658
 4.658  5.2115  5.2115  5.2115  5.2115     5.2115  5.2115  5.2115  4.658
 4.658  5.2115  5.2115  5.2115  5.2115     5.2115  5.2115  5.2115  4.658
 4.658  5.21

In [61]:
activ2 = similar(conv2out)
GeneralNN.relu!(activ2, conv2out)

18×18×1 Array{Float64, 3}:
[:, :, 1] =
 4.166  4.658   4.658   4.658   4.658   …  4.658   4.658   4.658   4.166
 4.658  5.2115  5.2115  5.2115  5.2115     5.2115  5.2115  5.2115  4.658
 4.658  5.2115  5.2115  5.2115  5.2115     5.2115  5.2115  5.2115  4.658
 4.658  5.2115  5.2115  5.2115  5.2115     5.2115  5.2115  5.2115  4.658
 4.658  5.2115  5.2115  5.2115  5.2115     5.2115  5.2115  5.2115  4.658
 4.658  5.2115  5.2115  5.2115  5.2115  …  5.2115  5.2115  5.2115  4.658
 4.658  5.2115  5.2115  5.2115  5.2115     5.2115  5.2115  5.2115  4.658
 4.658  5.2115  5.2115  5.2115  5.2115     5.2115  5.2115  5.2115  4.658
 4.658  5.2115  5.2115  5.2115  5.2115     5.2115  5.2115  5.2115  4.658
 4.658  5.2115  5.2115  5.2115  5.2115     5.2115  5.2115  5.2115  4.658
 4.658  5.2115  5.2115  5.2115  5.2115  …  5.2115  5.2115  5.2115  4.658
 4.658  5.2115  5.2115  5.2115  5.2115     5.2115  5.2115  5.2115  4.658
 4.658  5.2115  5.2115  5.2115  5.2115     5.2115  5.2115  5.2115  4.658
 4.658  5.21

In [62]:
locs = pooling(activ2)[2]

9×9×1 Array{CartesianIndex{2}, 3}:
[:, :, 1] =
 CartesianIndex(2, 2)  CartesianIndex(2, 1)  …  CartesianIndex(2, 1)
 CartesianIndex(1, 2)  CartesianIndex(1, 1)     CartesianIndex(1, 1)
 CartesianIndex(1, 2)  CartesianIndex(1, 1)     CartesianIndex(1, 1)
 CartesianIndex(1, 2)  CartesianIndex(1, 1)     CartesianIndex(1, 1)
 CartesianIndex(1, 2)  CartesianIndex(1, 1)     CartesianIndex(1, 1)
 CartesianIndex(1, 2)  CartesianIndex(1, 1)  …  CartesianIndex(1, 1)
 CartesianIndex(1, 2)  CartesianIndex(1, 1)     CartesianIndex(1, 1)
 CartesianIndex(1, 2)  CartesianIndex(1, 1)     CartesianIndex(1, 1)
 CartesianIndex(1, 2)  CartesianIndex(1, 1)     CartesianIndex(1, 1)

In [63]:
#pre-allocate pooling output
activ3 = zeros(9,9,5)
activ3 .= pooling(activ2)[1]

9×9×5 Array{Float64, 3}:
[:, :, 1] =
 5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115
 5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115
 5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115
 5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115
 5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115
 5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115
 5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115
 5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115
 5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115

[:, :, 2] =
 5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115
 5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115
 5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115
 5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115  5.2115
 5.2115  5.211

In [64]:
size(activ2)

(18, 18, 1)

In [65]:
activ4 = flatten_img(activ3)

405-element Vector{Float64}:
 5.211499999999994
 5.211499999999994
 5.211499999999994
 5.211499999999994
 5.211499999999994
 5.211499999999994
 5.211499999999994
 5.211499999999994
 5.211499999999994
 5.211499999999994
 ⋮
 5.211499999999994
 5.211499999999994
 5.211499999999994
 5.211499999999994
 5.211499999999994
 5.211499999999994
 5.211499999999994
 5.211499999999994
 5.211499999999994

In [66]:
size(activ4)

(405,)

In [67]:
wgts5 = rand(200,405)

200×405 Matrix{Float64}:
 0.264564   0.499394   0.211401   …  0.0946707  0.546875   0.2166
 0.360096   0.0500937  0.296936      0.41763    0.256018   0.574353
 0.657589   0.696926   0.413935      0.189194   0.668487   0.913756
 0.300441   0.016078   0.786409      0.258176   0.010706   0.757573
 0.878701   0.584112   0.325999      0.478353   0.504818   0.819486
 0.15461    0.892764   0.69506    …  0.700775   0.977408   0.856586
 0.892249   0.863775   0.942808      0.244984   0.0878049  0.247658
 0.516092   0.59022    0.506339      0.645829   0.108018   0.742264
 0.0928593  0.186991   0.907803      0.835136   0.451242   0.0312118
 0.85635    0.0953636  0.28143       0.49357    0.190795   0.903564
 ⋮                                ⋱                        
 0.105336   0.484056   0.316923      0.0550359  0.899936   0.00273768
 0.0917554  0.352044   0.759464      0.322118   0.257106   0.260723
 0.087632   0.605426   0.813543      0.101973   0.905544   0.912367
 0.127454   0.922015   0.84840

In [68]:
activ5 = wgts5 * activ4
activ5 = GeneralNN.relu!(activ5, activ5)

200-element Vector{Float64}:
 1052.0862910604676
 1011.4833408063458
 1084.1459900668553
 1024.6354670783398
 1086.4241780945267
 1038.9459102673502
 1086.9737726409119
 1103.462243652272
 1033.7556972776943
 1035.4419154719337
    ⋮
 1004.0477709528905
 1005.0129106627301
 1066.7827702306922
 1070.5783707606274
 1028.2981851651552
 1027.8845170687612
 1094.8063374963904
 1122.4398702980513
 1061.9825218070864

In [69]:
wgts6 = rand(10,200)

10×200 Matrix{Float64}:
 0.331552    0.572976  0.29725    0.418165   …  0.890169  0.447527  0.682387
 0.467751    0.738658  0.829962   0.449043      0.285527  0.741934  0.783015
 0.64425     0.801554  0.574983   0.790337      0.623312  0.737375  0.248729
 0.220639    0.711659  0.0266313  0.481491      0.930344  0.460203  0.539892
 0.383283    0.757457  0.0145169  0.132569      0.368832  0.951775  0.910547
 0.00137198  0.680266  0.584261   0.683982   …  0.255577  0.734059  0.0386203
 0.0351276   0.306318  0.301701   0.488961      0.282703  0.366925  0.861215
 0.504309    0.307854  0.109326   0.139228      0.158891  0.807588  0.410015
 0.357587    0.939992  0.808697   0.0419406     0.629862  0.888908  0.565451
 0.950075    0.457012  0.487158   0.378779      0.968253  0.343991  0.22486

In [70]:
z6 = wgts6 * activ5
z6 = reshape(z6,10,1)

10×1 Matrix{Float64}:
 111535.77016238925
 100882.71476227857
 103032.71764570678
 106515.37446925184
 106819.78752560157
 105215.48529494154
 109978.42234141764
 102931.17993604731
 103339.42582091196
 107316.85051862412

In [71]:
preds = rand(10,1)

10×1 Matrix{Float64}:
 0.5018134128482533
 0.7253681205311583
 0.35307149809256655
 0.20716418192349328
 0.07554750538846955
 0.06488128197503651
 0.22672128906122768
 0.15235918604372478
 0.693324586986274
 0.7588588803494144

In [72]:
GeneralNN.softmax!(preds,z6)

10×1 Matrix{Float64}:
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1

In [73]:
tst = rand(17,17,7);

In [74]:
dopad(tst, (1,1,1,1))

19×19×7 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0         0.0        0.0        …  0.0        0.0        0.0
 0.0  0.0219884   0.201466   0.320216      0.726499   0.438991   0.0
 0.0  0.352457    0.893201   0.372787      0.969242   0.073189   0.0
 0.0  0.572948    0.910954   0.531168      0.147482   0.0486037  0.0
 0.0  0.013658    0.708968   0.145765      0.142169   0.516014   0.0
 0.0  0.0448348   0.743023   0.413351   …  0.0516937  0.293739   0.0
 0.0  0.00998208  0.994724   0.280085      0.575419   0.201702   0.0
 0.0  0.53584     0.106638   0.216921      0.73854    0.423864   0.0
 0.0  0.356536    0.0961151  0.362956      0.727439   0.238867   0.0
 0.0  0.188512    0.449408   0.626606      0.365238   0.530237   0.0
 0.0  0.824555    0.0908351  0.10996    …  0.97494    0.611897   0.0
 0.0  0.212892    0.962419   0.646056      0.46545    0.420552   0.0
 0.0  0.176541    0.28337    0.0160501     0.336281   0.973338   0.0
 0.0  0.0431574   0.952964   0.791317      0.366845   0.798083  